# Barbell weight combinations

I have a bunch of 1" barbell plates that I want to use for lifting. I need to see what combinations are possible with these weights, so that I can determine if I need to purchase more, and if so, which to purchase.

We can solve this problem by representing it as a linear program.

In [1]:
using JuMP, GLPK
using DataFrames
using LinearAlgebra
using Dates

In [2]:
ENV["LINES"] = 1000;    # so that the DataFrames display all rows

I catalogued my collection of plates.

In [3]:
plates = DataFrame(weight=Float64[], quantity=Int[], material=Symbol[], thickness=Float64[])
push!(plates, (10, 4, :metal, 1+(1/8)))
push!(plates, (3, 4, :metal, 13/16))
push!(plates, (5, 6, :metal, 15/16))
push!(plates, (4.4, 2, :sand, 1+(13/16)))
push!(plates, (14.3, 4, :sand, 2+(1/4)))
push!(plates, (8.8, 4, :sand, 2))

#push!(plates, (45, 4, :metal, 1.3))   # I don't have these yet.

plates.pair_weight = plates.weight .* 2
plates.pair_quantity = plates.quantity .÷ 2

plates

,weight,quantity,material,thickness,pair_weight,pair_quantity
,Float64,Int64,Symbol,Float64,Float64,Int64
1,10.0,4,metal,1.125,20.0,2
2,3.0,4,metal,0.8125,6.0,2
3,5.0,6,metal,0.9375,10.0,3
4,4.4,2,sand,1.8125,8.8,1
5,14.3,4,sand,2.25,28.6,2
6,8.8,4,sand,2.0,17.6,2


The bar weighs 12 lb.

In [4]:
bar_weight = 12

12

In [5]:
model = Model(GLPK.Optimizer);

We need one decision variable for each quantity of plate pairs we load onto the bar. I.e. one variable for quantity of 10lb plate pairs, another variable for quantity of 5lb plate pairs, etc.

In [6]:
@variable(model, p[eachindex(plates.pair_weight)], Int)

6-element Vector{VariableRef}:
 p[1]
 p[2]
 p[3]
 p[4]
 p[5]
 p[6]

We need to set bounds on each variable, so that the solver doesn't try to load more plates than we own.

In [7]:
@constraint(model, 0 .<= p .<= plates.pair_quantity)

6-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.Interval{Float64}}, ScalarShape}}:
 p[1] ∈ [0.0, 2.0]
 p[2] ∈ [0.0, 2.0]
 p[3] ∈ [0.0, 3.0]
 p[4] ∈ [0.0, 1.0]
 p[5] ∈ [0.0, 2.0]
 p[6] ∈ [0.0, 2.0]

We can give the solver some leeway in finding a loading, by allowing a margin for its resulting weight. E.g. it might not be able to find a loading that matches our target exactly, but it might be able to find one within ± margin of our target.

In [8]:
@constraint(model, load_min, bar_weight + dot(plates.pair_weight, p) >= 0) 

load_min : 20 p[1] + 6 p[2] + 10 p[3] + 8.8 p[4] + 28.6 p[5] + 17.6 p[6] ≥ -12.0

In [9]:
@constraint(model, load_max, bar_weight + dot(plates.pair_weight, p) <= 0)

load_max : 20 p[1] + 6 p[2] + 10 p[3] + 8.8 p[4] + 28.6 p[5] + 17.6 p[6] ≤ -12.0

Our bar only has 9.75" of sleeve length on each end, so we need to make sure the solver only finds combinations that don't exceed this length.

In [10]:
@constraint(model, dot(2 .* plates.thickness, p) <=  2 * 9+(3/4))

2.25 p[1] + 1.625 p[2] + 1.875 p[3] + 3.625 p[4] + 4.5 p[5] + 4 p[6] ≤ 18.75

Our objective is to find a combination of as few plates as possible in order to match our desired loading.

In [11]:
@objective(model, Min, sum(p))

p[1] + p[2] + p[3] + p[4] + p[5] + p[6]

Finally, we run the solver for every progression increment between our starting weight and the total amount of weight we own, and we save the results to a DataFrame. If the solver can't find a loading, we record that it's missing.

In [12]:
starting_weight = 45
progression = 5

loadings = starting_weight:progression:dot(plates.pair_weight, plates.pair_quantity)

margin = 1

loading_results = DataFrame(
    target_weight = [],
    solver_weight = [],
    plate_pair_counts = []
)

for loading in loadings
    set_normalized_rhs(load_min, loading-margin)
    set_normalized_rhs(load_max, loading+margin)
    optimize!(model);
    push!(loading_results,
            (
                loading,
                termination_status(model)==MOI.OPTIMAL ? dot(plates.pair_weight, value.(p)) : missing,
                termination_status(model)==MOI.OPTIMAL ? transpose((Int64∘value).(p)) : missing
            )
        )
end

loading_results

,target_weight,solver_weight,plate_pair_counts
,Any,Any,Any
1,45.0,44.0,[0 0 0 1 0 2]
2,50.0,50.0,[2 0 1 0 0 0]
3,55.0,55.0,[0 0 0 1 1 1]
4,60.0,60.0,[2 0 2 0 0 0]
5,65.0,66.0,[0 0 0 1 2 0]
6,70.0,69.8,[0 1 0 0 1 2]
7,75.0,74.8,[0 0 0 0 2 1]
8,80.0,80.8,[0 1 0 0 2 1]
9,85.0,86.0,[1 0 0 1 2 0]


How many weeks of workouts can we get with the loadings the solver has determined?

In [13]:
loading_results |> dropmissing |> size |> (n -> divrem(n[1],3)) |> (wd -> 2*Week(wd[1]) + Day(wd[2]))

10 weeks, 1 day